# 任务 3：部署用于无服务器推理的模型

## 任务 3.1：环境设置

安装程序包和依赖项。

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import sys
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

将来自训练和优化实验的模型保存在默认 Amazon Simple Storage Service (Amazon S3) 存储桶中。使用 **create_model** 设置模型，并配置 **ModelDataUrl** 以引用经过训练的模型。

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## 任务 3.2：从提供的经过合成和重新训练的模型创建终端节点

Amazon SageMaker Serverless Inference 是一项专用推理功能，可帮助您部署和扩展机器学习 (ML) 模型。Serverless Inference 非常适合流量突增之间有空闲期并且可以容忍冷启动的工作负载。无服务器终端节点会自动启动计算资源，并根据流量进行缩减或扩展。因此，您无需选择实例类型或管理扩展策略。这免去了选择和管理服务器这样无差别的繁重工作。Serverless Inference 集成了 AWS Lambda，提供高可用性、内置容错能力和弹性伸缩。

使用 Amazon SageMaker Python SDK 创建无服务器终端节点分三个步骤。这些步骤与用于实时终端节点的步骤相同，但两者的配置不同：
1.在 SageMaker 中创建 SageMaker 模型。
2.为 HTTPS 终端节点创建终端节点配置。
3.创建 HTTPS 终端节点。

您已创建模型。现在，您可以创建终端节点配置和终端节点。

首先，设置终端节点配置名称和要使用的内存大小。然后，调用 CreateEndpointConfig API。

要创建终端节点配置，您需要设置以下选项：
- **VariantName**：生产变体（生产环境中的一个或多个模型）的名称。
- **ModelName**：您要托管的模型的名称。这是您在创建模型时指定的名称。
- **ServerlessConfig**：这是将终端节点设置为无服务器终端节点的位置。请为 **MemorySizeInMB** 和 **MaxConcurrency** 配置值。
    - **MemorySizeInMB**：分配的内存大小（1024MB、2048MB、3072MB、4096MB、5120MB 或 6144MB）。
    - **MaxConcurrency**：并发调用数（1 到 200）。

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-serverless-endpoint-{}'.format(create_date)                              

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant
            "ServerlessConfig": {
                "MemorySizeInMB": 2048, # The memory size
                "MaxConcurrency": 20 # Number of concurrent invocations
            }
        } 
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

接下来，创建终端节点。当您创建无服务器终端节点时，SageMaker 会为您预置和管理计算资源。然后，您可以向终端节点发出推理请求，并在响应中收到模型预测。SageMaker 会根据需要扩展和缩减计算资源以处理您的请求流量，而您只需为使用的资源付费。

您可以选择 SageMaker 提供的容器，也可以使用自己的容器。无服务器终端节点的 RAM 大小最小为 1024MB，最大为 6144MB。Serverless Inference 会自动分配与您选择的内存成正比的计算资源。

当终端节点正在运行时，辅助函数将打印终端节点 Amazon Resource Name (ARN)。终端节点可能需要长达 7 分钟的时间才能完成创建。

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


在 SageMaker Studio 中，您可以在 **Endpoints**（终端节点）选项卡下查看终端节点详细信息。

下一个步骤将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **serverless_inference.ipynb** 选项卡拖到一边，或者选择（右键单击）**serverless_inference.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览终端节点时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完终端节点后，通过选择 **serverless_inference.ipynb** 选项卡返回至笔记本。

1.选择 **SageMaker Home**（SageMaker 主页）图标。
2.选择 **Deployments**（部署）。
3.选择 **Endpoints**（终端节点）。

此时 SageMaker Studio 将显示 **Endpoints**（终端节点）选项卡。

4.选择 **Name**（名称）列中包含 **income-model-serverless-** 的终端节点。

如果终端节点未出现，请选择刷新图标，直到终端节点出现。

此时 SageMaker Studio 将显示 **ENDPOINT DETAILS**（终端节点详细信息）选项卡。

5.选择 **AWS settings**（AWS 设置）选项卡。

如果您在终端节点完成创建之前打开了终端节点，请选择刷新图标，直到 **Endpoint status**（终端节点状态）从 *Creating*（正在创建）变为 *InService*（正在运行）。

**Endpoint type**（终端节点类型）显示为 **Serverless**（无服务器）。**Endpoint runtime settings**（终端节点运行时设置）部分会显示您之前在笔记本中选择的配置。

## 任务 3.3：使用客户记录调用终端节点以进行无服务器推理

使用 SageMaker 托管服务部署模型后，您可以向终端节点发送测试数据，以便在其上测试模型。

如果终端节点在一段时间内没有收到流量，然后突然收到了新请求，则可能需要一些时间来启动计算资源以处理请求。这称为冷启动。由于无服务器终端节点按需预置计算资源，因此您的终端节点可能会进行冷启动。如果并发请求数超过了当前并发请求使用率，也可能会发生冷启动。冷启动时间取决于模型大小、下载模型所需的时间以及容器的启动时间。

有关无服务器推理和冷启动如何运作的更多信息，请参阅 [Serverless Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html)。

您又收到了几条客户记录。请使用一组收入值为 1 的记录和一组收入值为 0 的记录调用终端节点，以确认终端节点是否正常运作。此时系统将输出每条记录的预测分数的列表。

In [ ]:
#invoke-endpoint-serverless-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('47,0,4,9,0,3,4,0,1,0,1902,60\n' +
                '53,0,0,0,0,2,4,0,1,0,0,40\n' +
                '44,0,0,0,2,0,1,0,1,14344,0,40\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

start_time = time.time()
response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,1,2,1,0,0,0,35\n' +
                '56,2,1,1,0,1,0,0,0,0,0,50\n' +
                '61,2,0,0,0,0,0,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## 任务 3.4：删除终端节点

清理终端节点可以分三个步骤完成。首先，删除终端节点。然后，删除终端节点配置。最后，如果不再需要部署的模型，则删除该模型。

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### 总结

恭喜！ 您已使用 SageMaker 成功创建一个无服务器终端节点，并使用 SageMaker Python SDK 调用了该终端节点。

本实验的下一项任务侧重于部署使用异步推理进行推理的模型。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续执行**任务 4：部署用于异步推理的模型**。